In [1]:
!pip install -U cufflinks

Requirement already up-to-date: cufflinks in /usr/local/lib/python3.6/dist-packages (0.15)


In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import functools

In [3]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 1286 (delta 37), reused 43 (delta 19), pack-reused 1221
Receiving objects: 100% (1286/1286), 17.96 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (770/770), done.


In [4]:
BATCH_SIZE = 512 #@param ["512", "256", "128"] {type:"raw"}
MOMENTUM = 0.9 #@param ["0.9", "0.95", "0.975"] {type:"raw"}
WEIGHT_DECAY = 0.000125 #@param ["0.000125", "0.00025", "0.0005"] {type:"raw"}
LEARNING_RATE = 0.4 #@param ["0.4", "0.2", "0.1"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
WARMUP = 5 #@param {type:"slider", min:0, max:24, step:1}
BUCKET = 'gs://gs_colab'
PROJECT = 'tutorial3'

In [5]:
fw.colab_utils.setup_gcs()

In [6]:
fw.colab_utils.setup_kaggle_from_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [7]:
!kaggle $fw.datasets.URLs.KAGGLE_CIFAR10

  0% 0.00/3.04M [00:00<?, ?B/s]
100% 3.04M/3.04M [00:00<00:00, 47.4MB/s]
 99% 603M/610M [00:08<00:00, 126MB/s]
100% 610M/610M [00:08<00:00, 71.3MB/s]
 88% 92.0M/105M [00:00<00:00, 109MB/s] 
100% 105M/105M [00:00<00:00, 118MB/s] 
  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 62.1MB/s]


In [8]:
!apt install libarchive-dev
!pip install libarchive

data_dir_local = './data'
fw.io.unzip(['./train.7z', './test.7z'], data_dir_local)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libarchive-dev
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 470 kB of archives.
After this operation, 1,621 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libarchive-dev amd64 3.2.2-3.1ubuntu0.3 [470 kB]
Fetched 470 kB in 1s (602 kB/s)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
  Stored in director


INFO:tensorflow:Decompressing: /content/test.7z


In [9]:
data_dir_local_train = os.path.join(data_dir_local, 'train')
data_dir_local_test = os.path.join(data_dir_local, 'test')
fw.preprocess.check_rgb(data_dir_local_train, file_ext='png')
fw.preprocess.check_rgb(data_dir_local_test, file_ext='png')

In [10]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [11]:
local_train_fn = os.path.join(data_dir_local, "train.tfrec")

paths_train, y_train, labels = fw.data.data_dir_label_csv_tfrecord(
  data_dir=data_dir_local_train, csv_fn='./trainLabels.csv', output_fn=local_train_fn, file_ext='png')

In [12]:
n_train, n_classes = len(paths_train), len(labels)
n_train, n_classes

(50000, 10)

In [13]:
fw.plt.plot_counts_pie(y_train, labels, width=450)

In [14]:
fw.anim.show_image_files(paths_train)

In [15]:
X_train_mean, X_train_std, img_size, _ = fw.preprocess.compute_image_mean_std(local_train_fn, n_train, batch_size=100)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
train_fn = os.path.join(data_dir, "train.tfrec")
fw.io.upload_to_gcs(local_train_fn, train_fn)

INFO:tensorflow:Output file already exists. Skipping.


In [17]:
test_fn = os.path.join(data_dir, "test.tfrec")

path_test = fw.data.data_dir_no_label_tfrecord(data_dir=data_dir_local_test, output_fn=test_fn, file_ext='png')

n_test = len(path_test)
n_test

INFO:tensorflow:Output file already exists. Skipping.


300000

In [18]:
train_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_pad_crop(4),
  fw.transform.tfm_random_flip(),
  fw.transform.tfm_cutout(8, 8),
  fw.transform.tfm_set_shape(img_size, img_size),
]

test_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_set_shape(img_size, img_size),
]

parser_train = fw.data.get_tfexample_image_parser(train_tfms)
parser_test = fw.data.get_tfexample_image_parser(test_tfms, has_label=False)

In [19]:
train_input_func = lambda params: fw.data.tfrecord_ds(train_fn, parser_train, batch_size=params['batch_size'], training=True)
test_input_func = lambda params: fw.data.tfrecord_ds(test_fn, parser_test, batch_size=params['batch_size'], training=False)

In [20]:
def build_nn(c=64, weight=0.125):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*2, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*8, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(fw.layers.Classifier(n_classes, kernel_initializer=fw.layers.init_pytorch, weight=weight))
  return model

In [21]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS

In [22]:
lr_func = fw.train.triangular_lr(LEARNING_RATE/BATCH_SIZE, total_steps, warmup_steps=WARMUP*steps_per_epoch)
fw.plt.plot_lr_func(lr_func, total_steps)

In [23]:
opt_func = fw.train.sgd_optimizer(lr_func, mom=MOMENTUM, wd=WEIGHT_DECAY*BATCH_SIZE)
model_func = fw.train.get_clf_model_func(build_nn, opt_func, reduction=tf.losses.Reduction.SUM)

In [24]:
est = fw.train.get_tpu_estimator(steps_per_epoch, model_func, work_dir, trn_bs=BATCH_SIZE, val_bs=10000, pred_bs=10000)
est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tutorial3/2019-05-08-15:34:49', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.66.137.162:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f692d3a78d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.66.137.162:8470', '_evaluation_master': 'grpc://10.66.137.162:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=97, num_shards=N

In [25]:
y_preds = []
test_ids = []
for i, pred in enumerate(est.predict(test_input_func)):
  y_preds.append(pred['y_pred'])
  fn = os.path.basename(path_test[i])
  test_ids.append(fn[:-4])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/tutorial3/2019-05-08-15:34:49/model.ckpt-2328
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Deque

In [26]:
d = {'id': test_ids, 'label': [labels[y] for y in y_preds]}
df = pd.DataFrame(data=d)
df

,id,label
0,71397,frog
1,100309,truck
2,236626,deer
3,222834,horse
4,228659,horse
5,217449,dog
6,290172,dog
7,9186,bird
8,264529,dog
9,200691,deer


In [27]:
df.to_csv('submission.csv', index=False)

In [28]:
!kaggle competitions submit -c cifar-10 -f submission.csv -m "Tutorial 3"

100% 3.54M/3.54M [00:02<00:00, 1.80MB/s]
Successfully submitted to CIFAR-10 - Object Recognition in Images

In [31]:
!kaggle competitions submissions -c cifar-10

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
submission.csv  2019-05-08 15:36:23  Tutorial 3   complete  0.94000      0.94000       
submission.csv  2019-05-08 11:49:42  Tutorial 3   complete  0.94250      0.94250       
submission.csv  2019-05-02 17:01:11  Tutorial 3   complete  0.93970      0.93970       
submission.csv  2019-04-25 18:36:36  Tutorial 3   complete  0.94020      0.94020       
submission.csv  2019-04-21 18:15:04  Tutorial 3   complete  0.93770      0.93770       
submission.csv  2019-04-21 11:00:51  Tutorial 3   complete  0.93820      0.93820       
submission.csv  2019-04-21 10:49:18  from colab   complete  0.94110      0.94110       
submission.csv  2019-04-21 06:28:40  from colab   complete  0.10310      0.10310       
submission.csv  2019-04-21 06:25:01               complete  0.10310      0.10310       
submission.csv  2019-04-21 06:23

In [30]:
fw.io.create_clean_dir(work_dir)